# Test Experiments

In [18]:
import multiprocessing as mp
import pandas as pd
import numpy as np
import ampligraph
import requests
import rdflib
import torch
import os

In [19]:
torch.cuda.is_available()

True

In [20]:
def run_query(url, query):
    """Run SPARQL query and return results as a DataFrame."""
    headers = {
        "Accept": "application/sparql-results+json",
        "Content-Type": "application/sparql-query"
    }
    response = requests.post(url, data=query, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        cols = data['head']['vars']
        rows = [{col: row['value'] for col, row in item.items()} for item in data['results']['bindings']]
        return pd.DataFrame(rows, columns=cols)
    else:
        print(f"Error: {response.status_code}")
        return pd.DataFrame()

In [21]:
def download_file(url, path):
    """Download a file from the specified URL to the given path."""
    response = requests.get(url)
    return response
    if response.status_code == 200:
        with open(path, 'wb') as file:
            file.write(response.content)
    else:
        print(f"Error downloading file: {response.status_code}")

In [22]:
# SPARQL Endpoint URL
endpoint_url = "https://data.europa.eu/sparql"

# SPARQL Query
query = """
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX odp:  <http://data.europa.eu/euodp/ontologies/ec-odp#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT * WHERE { ?d a dcat:Dataset } LIMIT 1
"""

# Run the query
df = run_query(endpoint_url, query)

# Display the first few rows of the DataFrame
print(df.head())

                                                   d
0  http://data.europa.eu/88u/dataset/0800af55-8e5...


In [23]:
for uri in df['d']:
    print(uri)
    response = download_file(uri, "downloaded_dataset.csv")
    print(response.content)

http://data.europa.eu/88u/dataset/0800af55-8e56-49a0-8986-aa55151d0440
b'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width,initial-scale=1.0">\n  <title>data.europa.eu</title>\n  <!-- Import Favicon -->\n  <link rel="shortcut icon" type="image/ico" href="/data/assets/favicon-7jgnUwDt.ico">\n  <!-- Import Loading Animation -->\n  <script type="module" crossorigin src="/data/app.PfalbxMJ.js"></script>\n  <link rel="stylesheet" crossorigin href="/data/assets/index-vNvHa2_I.css">\n</head>\n<body>\n<div id="app">\n  <div class="spinner-container" id="good">\n    <div class="spinner"> </div>\n  </div>\n</div>\n</body>\n\n</html>\n'
